In [10]:
import getpass
import os

In [11]:
if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass.getpass("Enter API KEY")

Enter API KEY ········


In [3]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_groq import ChatGroq

In [13]:
llm = ChatGroq(model = 'mixtral-8x7b-32768',
              temperature = 0,
              max_tokens = 500,
              timeout = None,
              max_retries = 2
              )

In [14]:
messages = [
    ('system', 'You are a knowledgable assistant specialized in astrology'),
    ('human', 'What is a meteor?')
]

In [15]:
ai = llm.invoke(messages)
ai

AIMessage(content='I\'m here to help! While I\'m an assistant with a broad range of knowledge, I should clarify that I don\'t specialize in astrology. Astrology is the study of the positions and movements of celestial bodies and their supposed influence on human affairs and natural events. However, I can certainly provide information about meteors!\n\nA meteor, often referred to as a "shooting star," is a small piece of interplanetary matter that burns up upon entering the Earth\'s atmosphere. The friction caused by the meteor\'s high speed and the atmospheric gases produces a streak of light in the sky, which we see from the ground. Most meteors are small, about the size of a grain of sand, and they disintegrate completely before reaching the Earth\'s surface. Larger meteors that survive their passage through the atmosphere and reach the ground are called meteorites.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 205, 'prompt_tokens': 28, 'total_tokens

In [16]:
ai.content

'I\'m here to help! While I\'m an assistant with a broad range of knowledge, I should clarify that I don\'t specialize in astrology. Astrology is the study of the positions and movements of celestial bodies and their supposed influence on human affairs and natural events. However, I can certainly provide information about meteors!\n\nA meteor, often referred to as a "shooting star," is a small piece of interplanetary matter that burns up upon entering the Earth\'s atmosphere. The friction caused by the meteor\'s high speed and the atmospheric gases produces a streak of light in the sky, which we see from the ground. Most meteors are small, about the size of a grain of sand, and they disintegrate completely before reaching the Earth\'s surface. Larger meteors that survive their passage through the atmosphere and reach the ground are called meteorites.'

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
     (
         "system", 'You are a knowledgable assistant specialized in {specialization}'
     ) ,
        ("human", "{input}"),
    ]
)

chain = prompt | llm

answer = chain.invoke({
    "specialization": "Chemistry",
    "input": "What is a molecule?",
})

In [18]:
answer.content

"A molecule is the fundamental unit of a chemical compound. It is composed of two or more atoms that have chemically bonded together. The atoms can be the same (such as in O₂, a molecule of oxygen) or different (such as H₂O, a molecule of water, which consists of two hydrogen atoms and one oxygen atom).\n\nThe atoms in a molecule are held together by attractive forces called chemical bonds, which result from the sharing or transfer of electrons between the atoms. The arrangement of atoms in a molecule determines its chemical properties and structure.\n\nMolecules can range in size from very small, such as diatomic molecules like O₂ and H₂, to very large, such as proteins and DNA, which can consist of thousands or even millions of atoms.\n\nIn chemistry, the molecule is a basic unit of measurement, and the mole is a unit of amount of substance that contains exactly 6.022 x 10²³ molecules of a substance. This number is known as Avogadro's number."

In [19]:
#web scrapping

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43380?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
page_data

USER_AGENT environment variable not set, consider setting it to identify your requests.


"Bewirb dich für Software Engineer II\n\nJobsucheNavigation überspringenJobsucheNIKE, INC. JOBSBefristete JobsJoin The Talent CommunityUnser Leben bei NikeÜbersichtLeistungenMarkenÜbersichtJordanConverseTeamsÜbersichtAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public Affairs\r\nHuman ResourcesData & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyStandorteÜbersichtNike WHQNike New York HQEHQ: Hilversum, NiederlandeELC: Laakdal, BelgiumGreater China HQDiversität, Gleichstellung und InklusionÜbersichtMilitärBehinderung und InklusionInklusion der indigenen BevölkerungPraktikaTechnologySoftware Engineer IISalem, OregonWerde Teil des Teams 

In [35]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        It should only be in English
        ### VALID JSON (NO PREAMBLE):   
        """
)


chain_extract = prompt_extract | llm

res = chain_extract.invoke(input = {'page_data': page_data})

type(res.content)

type(eval(res.content))

list

In [40]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res[0])

{'role': 'Software Engineer II', 'experience': 'Experience in a professional setting', 'skills': ['Swift', 'Grand Central Dispatch (GCD)', 'UIKit', 'XCTest', 'Quick / Nimble', 'Xcode', 'MVVM Architecture', 'Memory Management & Retain Cycles', 'RESTful APIs', 'JSON parsing and data modeling', 'Cocoapods', 'Git', 'New Relic & Analytics', 'Agile / Scrum Methodology', 'Peer Code Reviews', 'Technical Documentation', 'SwiftUI (nice to have)', 'Objective-C (nice to have)'], 'description': 'SIM iOS is used in over 900 Nike-Owned and Nike Partner-Owned stores globally. It enables our Retail Athletes to perform the operational tasks needed to get Nike Inventory in front of Retail Consumers including Receiving, Binning and Stockroom Management, Sales floor replenishment, Auditing retail inventory, and Inventory adjustments which sync to SAP and Supply Hub to enable Finance and support Nike’s O2O strategy.'}


In [42]:
type(json_res[0])

dict

In [43]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [44]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore') #stores data on disk
collection = client.get_or_create_collection(name = "portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents = row["Techstack"],
                      metadatas = {"links": row["Links"]},
                      ids = [str(uuid.uuid4())])

In [48]:
res = collection.query(query_texts = job["skills"], n_results = 2).get('metadatas')
res


[[{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'}

In [47]:
job = json_res[0]
job["skills"]

['Swift',
 'Grand Central Dispatch (GCD)',
 'UIKit',
 'XCTest',
 'Quick / Nimble',
 'Xcode',
 'MVVM Architecture',
 'Memory Management & Retain Cycles',
 'RESTful APIs',
 'JSON parsing and data modeling',
 'Cocoapods',
 'Git',
 'New Relic & Analytics',
 'Agile / Scrum Methodology',
 'Peer Code Reviews',
 'Technical Documentation',
 'SwiftUI (nice to have)',
 'Objective-C (nice to have)']

In [49]:
prompt_email = PromptTemplate.from_template(
    """
     ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Zeb, a business development executive at ATC group. ATC is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ATC 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ATC's portfolio: {link_list}
        Remember you are Zeb, BDE at ATC. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
    """
)

In [52]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": "links"})
res.content

"Subject: Expert Software Engineering Solutions for SIM iOS at Nike\n\nDear Hiring Manager,\n\nI hope this email finds you well. I came across the Software Engineer II position for SIM iOS at Nike and believe that ATC Group can provide the expertise and innovative solutions to fulfill your needs.\n\nATC Group has extensive experience in developing and integrating custom software solutions for various industries, including retail. Our proficiency in Swift, Grand Central Dispatch (GCD), UIKit, XCTest, Quick / Nimble, Xcode, MVVM Architecture, Memory Management & Retain Cycles, RESTful APIs, JSON parsing and data modeling, Cocoapods, Git, New Relic & Analytics, Agile / Scrum Methodology, Peer Code Reviews, and Technical Documentation align perfectly with the required skills for this role.\n\nTo showcase our capabilities, I would like to highlight some relevant projects in our portfolio:\n\n1. [Project 1 - Retail Inventory Management System](links): Our team developed a comprehensive retai